<a href="https://colab.research.google.com/github/yuukun123/Python/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadstat
from google.colab import drive

In [7]:
# Tùy chỉnh để hiển thị biểu đồ đẹp hơn
sns.set(style="whitegrid")

In [9]:
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
# khai báo biến với file dữ liệu
data1 = "/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/DelawarePainDatabase_SecondaryDatabase_FaceGenCharacterization.xlsx"
df = pd.read_excel(data1)

# Đọc file .sav
df1, meta = pyreadstat.read_sav('/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/DelawarePainDatabase_StimulusCharacterization_forOSF.sav')

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [12]:
# xem thông tin dữ liệu
df.info()
# xem những hàng đầu tiên
df.head()

df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Expression       33 non-null     object 
 1   Norming Survey   33 non-null     object 
 2   PainRating       33 non-null     float64
 3   FearRating       33 non-null     float64
 4   AngerRating      33 non-null     float64
 5   DisgustRating    33 non-null     float64
 6   HappinessRating  33 non-null     float64
 7   SadnessRating    33 non-null     float64
 8   SurpriseRating   33 non-null     float64
 9   ThreatRating     22 non-null     object 
 10  ConfusionRating  32 non-null     object 
 11  FearComp         33 non-null     float64
 12  AngerComp        33 non-null     float64
 13  DisgustComp      33 non-null     float64
 14  HappinessComp    33 non-null     float64
 15  SadnessComp      33 non-null     float64
 16  SurpriseComp     33 non-null     float64
 17  ThreatComp       2

,Target,Consent,Race,Gender,Masculine_Neutral,Feminine_Neutral,BabyFaced_Neutral,Trustworthy_Neutral,Dominant_Neutral,Attractive_Neutral,...,AU12_OF,AU14_OF,AU15_OF,AU17_OF,AU20_OF,AU23_OF,AU25_OF,AU26_OF,AU28_OF,AU45_OF
0,DPD_1_LM1_p2c,full,Hispanic,Male,5.654545,1.763636,2.545455,3.709091,4.272727,3.345455,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,DPD_1_LM1_p3c,full,Hispanic,Male,5.654545,1.763636,2.545455,3.709091,4.272727,3.345455,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,DPD_1_LM1_p5c,full,Hispanic,Male,5.654545,1.763636,2.545455,3.709091,4.272727,3.345455,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
3,DPD_1_BM6_p4c,full,Black,Male,5.406780,1.711864,3.525424,3.508475,4.271186,3.084746,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,DPD_1_BM6_p6c,full,Black,Male,5.406780,1.711864,3.525424,3.508475,4.271186,3.084746,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


,Unnamed: 0,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,day,month,year,match_id,rec_id_base
0,0,1496,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,1.0,1.0,1970.0,1496,1496
4,4,1213,taylor,hathaway,7,yuranigh court,brentwood vlge,NaN,4220,nsw,1.0,1.0,1970.0,1213,1213
5,5,193,toby,webb,6,kent street,kersey' south,towradgi,3107,nsw,1.0,1.0,1970.0,193,193
17,17,729,andrew,klander,20,newman morris circuit,the willows,homebush,2285,vic,1.0,1.0,1970.0,729,729
19,19,1688,kasey,millar,0,burdekin avenue,crown allot,woollahra,4217,qld,1.0,1.0,1970.0,1688,1688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,4993,882,NaN,white,20,parker street,moy,calliope,4514,sa,1.0,1.0,1970.0,882,882
4994,4994,502,sarah,cionci,7,maccallum circuit,chateau royal,fish creek,6026,sa,1.0,1.0,1970.0,502,502
4995,4995,937,jack,campbell,169,marr street,rhosewyn,oakleigh,3356,vic,1.0,1.0,1970.0,937,937
4997,4997,1756,destynii,bowerman,12,halford crescent,sutton,nollamara,2431,qld,1.0,1.0,1970.0,1756,1756


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df.to_csv('/content/drive/MyDrive/titanic.csv', index=False)

Mounted at /content/drive
